# 预读取

In [2]:
import h5py
import cv2
import warp_norm
import matplotlib
# import sys
# sys.path.append("./FaceAlignment")
# import face_alignment
from skimage import io
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from model import gaze_network
from torchvision import transforms
import pickle
import pandas as pd
import copy
import os
from pt_module import StNet,StRefine
from ipdb import set_trace as st


In [3]:
colors = plt.cm.viridis(np.linspace(0, 1, 4))
model_dir = './ckpt'
state_name = 'spatical_transform_model_fake_eyetracking_dataset_1kg_128_72_error00_valid_random_ii_in_sequence_person_gt_lr_0.1_99_full.pt'
state_path = os.path.join(model_dir,state_name)
condition_label=['glass && upright tan','glass && upright chen',
                 'no glass && upright tan','no glass && upright chen',
                 'glass && not upright tan','glass && not upright chen',
                 'no glass && not upright tan','no glass && not upright chen',
                 'glass && indoor daylight tan', 'glass && indoor daylight chen',
                 'no glass && indoor daylight tan','no glass && indoor daylight chen',
                 'glass && only lamp tan','glass && only lamp chen',
                 'no glass && only lamp tan','no glass && only lamp chen',
                 'glass && only external lighting tan','glass && only external lighting chen',
                 'no glass && only external lighting tan','no glass && only external lighting chen',
                 'Normal indoor lighting at night && glass && no mask tan', 'Normal indoor lighting at night && glass && no mask chen',
                 'Normal indoor lighting at night && glass && large camera distance(70cm+) tan','Normal indoor lighting at night && glass && large camera distance(70cm+) chen',
                 'Normal indoor lighting at night && glass && middle camera distance(45-48cm) tan','Normal indoor lighting at night && glass && middle camera distance(45-48cm) chen',
                 'Normal indoor lighting at night && glass && small camera distance(32-35cm) tan','Normal indoor lighting at night && glass && small camera distance(32-35cm) chen',
                 'Normal indoor lighting at night && glass && large inclination angle(45°) tan','Normal indoor lighting at night && glass && large inclination angle(45°) chen',
                 'Normal indoor lighting at night && glass && middle inclination angle(30°) tan','Normal indoor lighting at night && glass && middle inclination angle(30°) chen',
                 'Normal indoor lighting at night && glass && small inclination angle(15°) tan','Normal indoor lighting at night && glass && small inclination angle(15°) chen',
                 'Normal indoor lighting at night && glass && multi device normal distance(1cm<x<10cm)gaze at the computer tan','Normal indoor lighting at night && glass && multi device normal distance(1cm<x<10cm)gaze at the computer chen',
                 'Normal indoor lighting at night && glass && multi device normal distance(1cm<x<10cm)gaze at the phone tan','Normal indoor lighting at night && glass && multi device normal distance(1cm<x<10cm)gaze at the phone chen',
                 'Normal indoor lighting at night && glass && multi device large distance(x<1cm)gaze at the computer tan','Normal indoor lighting at night && glass && multi device large distance(x<1cm)gaze at the computer chen',
                 'Normal indoor lighting at night && glass && multi device large distance(x<1cm)gaze at the phone tan','Normal indoor lighting at night && glass && multi device large distance(x<1cm)gaze at the phone chen',
                 'Normal indoor lighting at night && glass && multi device phone laid(x<1cm)gaze at the computer tan','Normal indoor lighting at night && glass && multi device phone laid(x<1cm)gaze at the computer chen',
                 'Normal indoor lighting at night && glass && multi device phone laid(x<1cm)gaze at the phone tan','Normal indoor lighting at night && glass && multi device phone laid(x<1cm)gaze at the phone chen']
print(len(condition_label))

46


In [4]:
# 读取数据
with open('/home/hgh/hghData/all_3_5.pkl', 'rb') as fo:
    datalist = pickle.load(fo, encoding='bytes')#que
with open('/home/hgh/hghData/gaze_pred_3_5.pkl', 'rb') as fo:
    predlist = pickle.load(fo, encoding='bytes')#que

In [5]:
# 提取datapath和pred
# print(predlist)
file_dict = []
preds = []
for pred in predlist:
    file_dict.append(int(os.path.splitext(os.path.basename(pred['org_data']))[0]))
    preds.append(pred['pred'])
preds = np.asarray(preds).reshape((-1,2))

print(file_dict)
print(preds)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [6]:
def get_condition_number(file_dict):
    if (1 <= file_dict <= 100):
        return 0
    if (101 <= file_dict <= 200):
        return 1
    if (201 <= file_dict <= 300):
        return 2
    if (301 <= file_dict <= 400):
        return 3
    if (401 <= file_dict <= 500):
        return 4
    if (501 <= file_dict <= 600):
        return 5
    if (601 <= file_dict <= 700):
        return 6
    if (701 <= file_dict <= 800):
        return 7
    if (801 <= file_dict <= 850):
        return 8
    if (851 <= file_dict <= 900):
        return 9
    if (901 <= file_dict <= 950):
        return 10
    if (951 <= file_dict <= 1000):
        return 11
    if (1001 <= file_dict <= 1050):
        return 12
    if (1051 <= file_dict <= 1100):
        return 13
    if (1101 <= file_dict <= 1150):
        return 14
    if (1151 <= file_dict <= 1200):
        return 15
    if (1201 <= file_dict <= 1250):
        return 16
    if (1251 <= file_dict <= 1300):
        return 17
    if (1301 <= file_dict <= 1350):
        return 18
    if (1351 <= file_dict <= 1400):
        return 19
    if (1401 <= file_dict <= 1420):
        return 20
    if (1421 <= file_dict <= 1440):
        return 21
    if (1441 <= file_dict <= 1520):
        return 22
    if (1521 <= file_dict <= 1600):
        return 23
    if (1601 <= file_dict <= 1620):
        return 24
    if (1621 <= file_dict <= 1640):
        return 25
    if (1641 <= file_dict <= 1700):
        return 26
    if (1701 <= file_dict <= 1760):
        return 27
    if (1761 <= file_dict <= 1780):
        return 28
    if (1781 <= file_dict <= 1800):
        return 29
    if (1801 <= file_dict <= 1820):
        return 30
    if (1821 <= file_dict <= 1840):
        return 31
    if (1841 <= file_dict <= 1900):
        return 32
    if (1901 <= file_dict <= 1960):
        return 33
    if (1961 <= file_dict <= 1980):
        return 34
    if (1981 <= file_dict <= 2000):
        return 35
    if (2001 <= file_dict <= 2050):
        return 36
    if (2051 <= file_dict <= 2100):
        return 37
    if (2101 <= file_dict <= 2150):
        return 38
    if (2151 <= file_dict <= 2200):
        return 39
    if (2201 <= file_dict <= 2250):
        return 40
    if (2251 <= file_dict <= 2300):
        return 41
    if (2301 <= file_dict <= 2350):
        return 42
    if (2351 <= file_dict <= 2400):
        return 43
    if (2401 <= file_dict <= 2450):
        return 44
    if (2451 <= file_dict <= 2500):
        return 45
    if (2501 <= file_dict <= 2550):
        return 46
    if (2551 <= file_dict <= 2600):
        return 47

In [7]:
data_dict = {}
pred_dict = {}
for filename, data, pred in zip(file_dict, datalist, predlist):
    condition = get_condition_number(filename)

    if condition not in data_dict:
        data_dict[condition] = []
        pred_dict[condition] = []
        
    data_dict[condition].append(data)
    pred_dict[condition].append(pred['pred'])

for i in range(len(pred_dict)):
    pred_dict[i] = np.vstack(pred_dict[i])
for condition, data_list in data_dict.items():
    print(f"Condition {condition}: {data_list}")
#print(pred_dict[0].shape)

Condition 0: [<gaze_normalize.GazeNormalize object at 0x7efc78309f10>, <gaze_normalize.GazeNormalize object at 0x7efe9ce66d00>, <gaze_normalize.GazeNormalize object at 0x7efeac31d700>, <gaze_normalize.GazeNormalize object at 0x7efc782b3040>, <gaze_normalize.GazeNormalize object at 0x7efc782b31c0>, <gaze_normalize.GazeNormalize object at 0x7efc782b33d0>, <gaze_normalize.GazeNormalize object at 0x7efc782b3550>, <gaze_normalize.GazeNormalize object at 0x7efc782b3700>, <gaze_normalize.GazeNormalize object at 0x7efc782b3880>, <gaze_normalize.GazeNormalize object at 0x7efc782b39d0>, <gaze_normalize.GazeNormalize object at 0x7efc782b3b50>, <gaze_normalize.GazeNormalize object at 0x7efc782b3ca0>, <gaze_normalize.GazeNormalize object at 0x7efc782b3e80>, <gaze_normalize.GazeNormalize object at 0x7efc782b3fd0>, <gaze_normalize.GazeNormalize object at 0x7efc782bc130>, <gaze_normalize.GazeNormalize object at 0x7efc782bc2b0>, <gaze_normalize.GazeNormalize object at 0x7efc782bc430>, <gaze_normalize.G

In [8]:
# 将pitchyaw转换成vector
# 在这个流程中不用还原注视向量
for i in range(len(pred_dict)):
    pred_dict[i] = warp_norm.pitchyaw_to_vector(pred_dict[i])

In [9]:
pixel_scale_tan = np.array([0.202, 0.224])
pixel_scale_chen = np.array([0.22, 0.235])

# 将注视向量转换成gc
# 未还原，因此人脸距离为600mm
gc_dict = {}
for i in range(len(pred_dict)):
    if i not in gc_dict:
        gc_dict[i] = []
    for vector in pred_dict[i]:
        if i%2 == 0:
            gc_dict[i].append(warp_norm.vector_to_gc(vector,pixel_scale_tan))
        else:
            gc_dict[i].append(warp_norm.vector_to_gc(vector,pixel_scale_chen))

print(gc_dict[i])

[array([617.39934045, 608.26899968]), array([694.17744083, 702.58446918]), array([ 923.31372911, 1104.90537382]), array([775.82750588, 730.69706012]), array([738.53698214, 849.31425048]), array([575.53587197, 709.16189776]), array([496.97554738, 735.01624092]), array([ 767.32732957, 1021.0313891 ]), array([ 715.28806988, 1038.31597796]), array([647.39006953, 939.92897457]), array([896.09608267, 876.270949  ]), array([931.56861912, 907.71329816]), array([1033.94148865,  974.20921991]), array([ 931.32211153, 1104.29323396]), array([1207.0818947 , 1353.38541375]), array([773.76482724, 939.41220583]), array([366.30998555, 873.1754791 ]), array([ 417.25902836, 1046.71965451]), array([824.80888523, 849.52261839]), array([858.16567275, 890.97207804]), array([334.18350664, 919.16154455]), array([ 872.51440192, 1008.04722036]), array([1059.36684757, 1028.67822419]), array([-746.69274782, 1191.12893205]), array([-789.12299322,  824.8250258 ]), array([-470.37460225,  869.95717974]), array([-504.6

In [10]:
org_tan = np.array([800,0])#tan 1600*825
org_chen = np.array([650,0])#chen 1300*720


pred_gc_org = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for i in range(46):
    if i%2 == 0:
        pred_gc_org[i] = org_tan + gc_dict[i]
    else:
        pred_gc_org[i] = org_chen + gc_dict[i]

In [2]:
# 查找注视点相近的点

# 计算所有点之间的距离
def calculate_distances(data_dict, threshold):
    close_pairs = []
    point_indices = []
    for i, group in enumerate(data_dict):
        for j, (data1) in enumerate(group):
            (x1,y1) = data1.label
            for k, (data2) in enumerate(group):
                (x2, y2) = data2.label
                if i != k:  # 避免比较同一组内的点
                    distance = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))
                    if distance <= threshold:
                        close_pairs.append((i, j, k))
                        point_indices.append((i, j, k, distance))
    return close_pairs, point_indices

# 设定阈值
threshold = 1.0  # 这个值可以根据实际情况调整

# print(data_dict)
# 找到接近的点
close_pairs, point_indices = calculate_distances(data_dict, threshold)

# 返回结果
print(len(close_pairs))

NameError: name 'data_dict' is not defined

In [ ]:
# 将归一化向量还原
for i in range(len(pred_dict)):
    org_pred[i].append(np.dot(np.linalg.inv(RMat[i][j]), pred[i][j].T))
